In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.impute import KNNImputer

import warnings
warnings.filterwarnings('ignore')

## Reading in Data and Gathering Initial Thoughts
- First thing we see when looking at the dataframe head is the use of commas in seemingly numeric values. Most likely these should be replaced with decimal points. We output the columns with commas in their values and based on the data descriptions file, we can determine these should all be numeric. We string replace the commas to decimal points on those columns and then convert them to floats.  
     - From the data descriptions file, we can see that there are more columns that will need to be converted to floats. Any column with 'mean', 'avg', 'price', 'tot', or 'adj' should be numeric. And of the remaining columns, we see that the numeric columns are already in the correct data type. We convert all applicable columns here together.
- Next we will quickly check for any duplicates and see that we have none.
- When we describe the dataframe, we see that `truck`, `rv`, and `forgntvl` are all numeric columns, but should be treated as categorical when dealing with null value imputation and outliers later on. We may need to convert them back to binary flags for our correlation and modeling later, but for now we will switch them to Y/N flags. 

In [2]:
raw_df = pd.read_csv('../data/raw/dataset.csv', sep=';')

In [3]:
print('DF Shape: ', raw_df.shape)

raw_df.head()

DF Shape:  (100000, 100)


,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,ovrrev_Mean,vceovr_Mean,datovr_Mean,roam_Mean,change_mou,...,forgntvl,ethnic,kid0_2,kid3_5,kid6_10,kid11_15,kid16_17,creditcd,eqpdays,Customer_ID
0,"23,9975","219,25","22,5","0,2475",0,0,0,0,0,"-157,25",...,0.0,N,U,U,U,U,U,Y,361.0,1000001
1,"57,4925","482,75","37,425","0,2475","22,75","9,1","9,1",0,0,"532,25",...,0.0,Z,U,U,U,U,U,Y,240.0,1000002
2,"16,99","10,25","16,99",0,0,0,0,0,0,"-4,25",...,0.0,N,U,Y,U,U,U,Y,1504.0,1000003
3,38,"7,5",38,0,0,0,0,0,0,"-1,5",...,0.0,U,Y,U,U,U,U,Y,1812.0,1000004
4,"55,23","570,5","71,98",0,0,0,0,0,0,"38,5",...,0.0,I,U,U,U,U,U,Y,434.0,1000005


In [4]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 100 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   rev_Mean          99643 non-null   object 
 1   mou_Mean          99643 non-null   object 
 2   totmrc_Mean       99643 non-null   object 
 3   da_Mean           99643 non-null   object 
 4   ovrmou_Mean       99643 non-null   object 
 5   ovrrev_Mean       99643 non-null   object 
 6   vceovr_Mean       99643 non-null   object 
 7   datovr_Mean       99643 non-null   object 
 8   roam_Mean         99643 non-null   object 
 9   change_mou        99109 non-null   object 
 10  change_rev        99109 non-null   object 
 11  drop_vce_Mean     100000 non-null  object 
 12  drop_dat_Mean     100000 non-null  object 
 13  blck_vce_Mean     100000 non-null  object 
 14  blck_dat_Mean     100000 non-null  object 
 15  unan_vce_Mean     100000 non-null  object 
 16  unan_dat_Mean     10

In [11]:
# percentage of null values

print(((raw_df.isnull().sum().sum()) / (raw_df.shape[0] * raw_df.shape[1])) * 100)

3.42969


In [5]:
# finding cols with commas and their data types

test_df = raw_df.map(str)
contains_comma = test_df.apply(lambda x: x.str.contains(','), axis=0).any()
columns_with_comma = contains_comma[contains_comma].index.tolist()

print('Columns with commas: ', columns_with_comma)

print('Data Types: ', raw_df[columns_with_comma].dtypes.unique())

Columns with commas:  ['rev_Mean', 'mou_Mean', 'totmrc_Mean', 'da_Mean', 'ovrmou_Mean', 'ovrrev_Mean', 'vceovr_Mean', 'datovr_Mean', 'roam_Mean', 'change_mou', 'change_rev', 'drop_vce_Mean', 'drop_dat_Mean', 'blck_vce_Mean', 'blck_dat_Mean', 'unan_vce_Mean', 'unan_dat_Mean', 'plcd_vce_Mean', 'plcd_dat_Mean', 'recv_vce_Mean', 'recv_sms_Mean', 'comp_vce_Mean', 'comp_dat_Mean', 'custcare_Mean', 'ccrndmou_Mean', 'cc_mou_Mean', 'inonemin_Mean', 'threeway_Mean', 'mou_cvce_Mean', 'mou_cdat_Mean', 'mou_rvce_Mean', 'owylis_vce_Mean', 'mouowylisv_Mean', 'iwylis_vce_Mean', 'mouiwylisv_Mean', 'peak_vce_Mean', 'peak_dat_Mean', 'mou_peav_Mean', 'mou_pead_Mean', 'opk_vce_Mean', 'opk_dat_Mean', 'mou_opkv_Mean', 'mou_opkd_Mean', 'drop_blk_Mean', 'attempt_Mean', 'complete_Mean', 'callfwdv_Mean', 'callwait_Mean', 'totmou', 'totrev', 'adjrev', 'adjmou', 'avgrev', 'avgmou', 'avgqty', 'hnd_price']
Data Types:  [dtype('O')]


In [6]:
# string replace ',' --> '.' 

decimal_df = raw_df.copy()

for col in columns_with_comma:
    if decimal_df[col].dtype == 'object':
        decimal_df[col] = decimal_df[col].astype(str)
        decimal_df[col] = decimal_df[col].str.replace(',', '.', regex=False)

decimal_df[columns_with_comma] = decimal_df[columns_with_comma].apply(pd.to_numeric, errors='coerce')

In [7]:
# get numeric cols

str_num_cols = [col for col in decimal_df.columns if any(substring in col.lower() for substring in ['mean', 'avg', 'price', 'tot', 'adj'])]

remaining_cols = decimal_df.columns.difference(columns_with_comma + str_num_cols)

print(remaining_cols)

Index(['Customer_ID', 'HHstatin', 'actvsubs', 'adults', 'area', 'asl_flag',
       'churn', 'crclscod', 'creditcd', 'dualband', 'dwllsize', 'dwlltype',
       'eqpdays', 'ethnic', 'forgntvl', 'hnd_webcap', 'income', 'infobase',
       'kid0_2', 'kid11_15', 'kid16_17', 'kid3_5', 'kid6_10', 'lor', 'marital',
       'models', 'months', 'new_cell', 'numbcars', 'ownrent', 'phones',
       'prizm_social_one', 'refurb_new', 'rv', 'truck', 'uniqsubs'],
      dtype='object')


In [8]:
# convert numeric cols

numeric_df = decimal_df.copy()

num_cols = list(set(columns_with_comma + str_num_cols))

numeric_df[num_cols] = numeric_df[num_cols].apply(pd.to_numeric)

In [9]:
# check for any duplicate rows

numeric_df.duplicated().sum()

0

In [10]:
numeric_df.describe()

,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,ovrrev_Mean,vceovr_Mean,datovr_Mean,roam_Mean,change_mou,...,models,truck,rv,lor,adults,income,numbcars,forgntvl,eqpdays,Customer_ID
count,99643.000000,99643.000000,99643.000000,99643.000000,99643.000000,99643.000000,99643.000000,99643.000000,99643.000000,99109.000000,...,99999.000000,98268.000000,98268.000000,69810.000000,76981.000000,74564.000000,50634.000000,98268.000000,99999.000000,1.000000e+05
mean,58.719985,513.559937,46.179136,0.888828,41.072247,13.559560,13.295062,0.261318,1.286405,-13.933818,...,1.545825,0.188820,0.082580,6.177238,2.530326,5.783112,1.567563,0.057974,391.932309,1.050000e+06
std,46.291677,525.168140,23.623489,2.177619,97.296150,30.500885,30.056089,3.126531,14.711374,276.087509,...,0.898395,0.391368,0.275248,4.735267,1.452819,2.182132,0.625456,0.233696,256.482193,2.886766e+04
min,-6.167500,0.000000,-26.915000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3875.000000,...,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,-5.000000,1.000001e+06
25%,33.260000,150.750000,30.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-87.000000,...,1.000000,0.000000,0.000000,2.000000,1.000000,4.000000,1.000000,0.000000,212.000000,1.025001e+06
50%,48.195000,355.500000,44.990000,0.247500,2.750000,1.000000,0.682500,0.000000,0.000000,-6.250000,...,1.000000,0.000000,0.000000,5.000000,2.000000,6.000000,1.000000,0.000000,342.000000,1.050000e+06
75%,70.750000,703.000000,59.990000,0.990000,42.000000,14.437500,14.025000,0.000000,0.235000,63.000000,...,2.000000,0.000000,0.000000,9.000000,3.000000,7.000000,2.000000,0.000000,530.000000,1.075000e+06
max,3843.262500,12206.750000,409.990000,159.390000,4320.750000,1102.400000,896.087500,423.540000,3685.200000,31219.250000,...,16.000000,1.000000,1.000000,15.000000,6.000000,9.000000,3.000000,1.000000,1823.000000,1.100000e+06


In [11]:
numeric_df[['truck', 'rv', 'forgntvl']] = numeric_df[['truck', 'rv', 'forgntvl']].replace({1.0: 'Y', 0.0: 'N'})

## Handling Null Values 

- We check for null values. We see from the info output that many of the features have some null values. We find the features with the highest null percentage are `numbcars`, `dwllsize`, `HHstatin`, `ownrent`, and `dwlltype`.
- Almost half of the values in `numbcars` are null. Since the company is in the telecommunications sector, we most likely do not need this variable because we can pick up on information such as income and number of adults in other features. Therefore, we will go ahead and drop this feature. 
- `dwllsize`, `HHstatin`, and `ownrent` all have over a third of their values being null. We can start by imputing the null values of `dwllsize` with the corresponding average size of `dwlltype`. Although, since `dwlltype` also is in the top five features with the most null values, this most likely will not improve `dwllsize` greatly. It handles ~6k records, so we still need to impute another way. For the remaining null values, we are going to impute them with the average amongst the most common factors for a house size: `income`, `area`, and `num_ppl_household`. The final feature is created by summing `adults`, `kid0_2`, `kid3_5`, `kid6_10`, `kid11_15`, and `kid16_17`. 
- `HHstatin` is a Premier household status indicator. Since we have other indicators for income and revenue for the customer, we will drop this feature. 
- We will use the same imputing logic that we used for `dwllsize` for `ownrent`. 
- For `dwlltype`, we will take the average value from `dwllsize`. 
- For the remaining features, we will impute with the mode or average, depending on data type. We do this since we will be trying regression models as part of our prediction modeling efforts, so we will not be able to include any null values in our data. 

In [18]:
# check the top features with most null

null_percentages = (numeric_df.isnull().mean() * 100).round(2).sort_values(ascending=False)

null_percentages.head(10)

numbcars      49.37
dwllsize      38.31
HHstatin      37.92
ownrent       33.71
dwlltype      31.91
lor           30.19
income        25.44
adults        23.02
infobase      22.08
hnd_webcap    10.19
dtype: float64

In [13]:
# drop features

null_df = numeric_df.drop(columns=['numbcars', 'HHstatin'])

In [14]:
# fill dwllsize with avg dwlltype

def mode_fillna(group):
    mode_value = group.mode().iloc[0] 
    return group.fillna(mode_value) 

null_df['dwllsize'] = null_df.groupby('dwlltype')['dwllsize'].apply(mode_fillna).reset_index(drop=True)

print('Remaining dwllsize nulls: ', null_df['dwllsize'].isnull().sum())

Remaining dwllsize nulls:  31909


In [15]:
# create num ppl feature

null_df['num_ppl_household'] = (null_df[['kid0_2', 'kid3_5', 'kid6_10', 'kid11_15', 'kid16_17']] == 'Y').sum(axis=1) + null_df['adults']

mask = pd.notnull(null_df['num_ppl_household'])
null_df.loc[mask, 'num_ppl_household'] = null_df.loc[mask, 'num_ppl_household'].astype(int)

In [55]:
mask = pd.notnull(null_df['income'])
null_df.loc[mask, 'income'] = null_df.loc[mask, 'income'].astype(int)

In [17]:
# fill dwllsize with avg across income, area, num_ppl_household

null_df[['income', 'area', 'num_ppl_household']] = null_df[['income', 'area', 'num_ppl_household']].fillna('NaN')

def mode_default(x):
    mode_values = x.mode()
    return mode_values.iat[0] if not mode_values.empty else None

mode_mapping = null_df.groupby(['income', 'area', 'num_ppl_household'])['dwllsize'].agg(mode_default).to_dict()
null_df['dwllsize'] = null_df.apply(lambda x: mode_mapping[(x['income'], x['area'], x['num_ppl_household'])] if pd.isnull(x['dwllsize']) else x['dwllsize'], axis=1)

null_df[['income', 'area', 'num_ppl_household']] = null_df[['income', 'area', 'num_ppl_household']].replace('NaN', np.nan)

print('Remaining dwllsize nulls: ', null_df['dwllsize'].isnull().sum())

Remaining dwllsize nulls:  73


In [19]:
# fill ownrent with avg across income, area, num_ppl_household

null_df[['income', 'area', 'num_ppl_household']] = null_df[['income', 'area', 'num_ppl_household']].fillna('NaN')

def mode_default(x):
    mode_values = x.mode()
    return mode_values.iat[0] if not mode_values.empty else None

mode_mapping = null_df.groupby(['income', 'area', 'num_ppl_household'])['ownrent'].agg(mode_default).to_dict()
null_df['ownrent'] = null_df.apply(lambda x: mode_mapping[(x['income'], x['area'], x['num_ppl_household'])] if pd.isnull(x['ownrent']) else x['ownrent'], axis=1)

null_df[['income', 'area', 'num_ppl_household']] = null_df[['income', 'area', 'num_ppl_household']].replace('NaN', np.nan)

print('Remaining ownrent nulls: ', null_df['ownrent'].isnull().sum())

Remaining ownrent nulls:  1117


In [20]:
# fill ownrent with avg across income, area

null_df[['income', 'area']] = null_df[['income', 'area']].fillna('NaN')

def mode_default(x):
    mode_values = x.mode()
    return mode_values.iat[0] if not mode_values.empty else None

mode_mapping = null_df.groupby(['income', 'area'])['ownrent'].agg(mode_default).to_dict()
null_df['ownrent'] = null_df.apply(lambda x: mode_mapping[(x['income'], x['area'])] if pd.isnull(x['ownrent']) else x['ownrent'], axis=1)

null_df[['income', 'area']] = null_df[['income', 'area']].replace('NaN', np.nan)

print('Remaining ownrent nulls: ', null_df['ownrent'].isnull().sum())

Remaining ownrent nulls:  9


In [21]:
# fill dwlltype with avg dwllsize

def mode_fillna(group):
    mode_value = group.mode().iloc[0] 
    return group.fillna(mode_value) 

null_df['dwlltype'] = null_df.groupby('dwllsize')['dwlltype'].apply(mode_fillna).reset_index(drop=True)

print('Remaining dwlltype nulls: ', null_df['dwlltype'].isnull().sum())

Remaining dwlltype nulls:  73


In [22]:
# impute remaining missing vals with avg / mode

numeric_cols = [col for col in null_df.columns if null_df[col].dtype in ['int64', 'float64']]
cat_cols = [col for col in null_df.columns if col not in numeric_cols]

for col in numeric_cols:
    null_df[col].fillna(null_df[col].mean(), inplace=True)

for col in cat_cols:
    null_df[col].fillna(null_df[col].mode()[0], inplace=True)

print('Total remaining nulls: ', null_df.isnull().sum().sum())

Total remaining nulls:  0


In [23]:
null_df['num_ppl_household'] = null_df['num_ppl_household'].astype(int)

## Exploring Outliers
- All the columns aside from `churn` and `Customer_ID` have outliers. Many of the features with too many outliers to simply drop them from the dataset without greatly impacting our data. 
- One option could be binning the values to reduce outliers. However, we want to try and keep these variables as numeric for modeling purposes later. Also, a technique like capping or trimming would help eliminate the outliers, but we should be hesitant to completely remove outlier data without having a deep understanding of the company and their industry.
- We try a simple linear regression for outlier imputation. While many features are now resolved, others seem to still have been imputed with outlier data. Therefore, we try a more sophisticated method in KNN outlier imputation. 
- KNN imputation did not solve our outlier issue. Because there are so many of them and the imputation techniques are not resolving them, we will move forward and keep this in mind for modeling. Something like scaling could be used to help the outliers, or models that are more robust and less sensitive to outliers. We will explore this later in the modeling section. 

In [24]:
# determining columns with most outliers

def count_outliers(df, numeric_columns):
    outliers_count = {}
    for column in numeric_columns:
        q1 = df[column].quantile(0.25)
        q3 = df[column].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        outliers_count[column] = df[(df[column] < lower_bound) | (df[column] > upper_bound)].shape[0]
    
    outliers_df = pd.DataFrame(list(outliers_count.items()), columns=['Feature', 'Number of Outliers']).sort_values(by='Number of Outliers', ascending=False)
    outliers_df['Outlier Percentage'] = outliers_df['Number of Outliers'] / (len(null_df)) * 100

    return outliers_df

outliers_table = count_outliers(null_df, numeric_cols)

outliers_table

,Feature,Number of Outliers,Outlier Percentage
10,change_rev,26325,26.325
8,roam_Mean,18736,18.736
18,plcd_dat_Mean,14980,14.980
47,callwait_Mean,14305,14.305
25,cc_mou_Mean,14265,14.265
...,...,...,...
20,recv_sms_Mean,872,0.872
46,callfwdv_Mean,433,0.433
67,hnd_price,254,0.254
48,churn,0,0.000


In [28]:
x = outliers_table.head(50)
x

,Feature,Number of Outliers,Outlier Percentage
10,change_rev,26325,26.325
8,roam_Mean,18736,18.736
18,plcd_dat_Mean,14980,14.980
47,callwait_Mean,14305,14.305
25,cc_mou_Mean,14265,14.265
7,datovr_Mean,14030,14.030
9,change_mou,13768,13.768
24,ccrndmou_Mean,13456,13.456
29,mou_cdat_Mean,13393,13.393
22,comp_dat_Mean,13393,13.393


In [48]:
# linear regression outlier imputation 

def regression_impute(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    non_outliers = series[(series >= lower_bound) & (series <= upper_bound)]
    X_train = non_outliers.index.to_numpy().reshape(-1, 1)
    y_train = non_outliers.to_numpy().reshape(-1, 1)

    model = LinearRegression()
    model.fit(X_train, y_train)

    outliers = series[(series < lower_bound) | (series > upper_bound)]

    if outliers.empty:
        return series
    
    X_outliers = outliers.index.to_numpy().reshape(-1, 1)
    y_pred = model.predict(X_outliers)

    series.loc[X_outliers.flatten()] = y_pred.flatten()
    
    return series

outliers_df = null_df.copy()
outliers_df[numeric_cols] = null_df[numeric_cols].apply(regression_impute)

In [49]:
count_outliers(outliers_df, numeric_cols)

,Feature,Number of Outliers,Outlier Percentage
71,adults,31102,31.102
10,change_rev,26669,26.669
72,income,18923,18.923
27,threeway_Mean,17901,17.901
25,cc_mou_Mean,12750,12.750
...,...,...,...
12,drop_dat_Mean,0,0.000
14,blck_dat_Mean,0,0.000
16,unan_dat_Mean,0,0.000
74,Customer_ID,0,0.000


In [50]:
# knn outlier imputation - start with lower k 

imputer = KNNImputer(n_neighbors=5)
knn_outlier_df = outliers_df.copy()
knn_outlier_df[numeric_cols] = imputer.fit_transform(knn_outlier_df[numeric_cols])

In [51]:
knn_outlier_df

,rev_Mean,mou_Mean,totmrc_Mean,da_Mean,ovrmou_Mean,ovrrev_Mean,vceovr_Mean,datovr_Mean,roam_Mean,change_mou,...,ethnic,kid0_2,kid3_5,kid6_10,kid11_15,kid16_17,creditcd,eqpdays,Customer_ID,num_ppl_household
0,23.997500,219.250000,22.500000,0.247500,0.000000,0.00000,0.000000,0.0,0.000000,-157.250000,...,N,U,U,U,U,U,Y,361.000000,1000001.0,1.0
1,57.492500,482.750000,37.425000,0.247500,22.750000,9.10000,9.100000,0.0,0.000000,-10.410104,...,Z,U,U,U,U,U,Y,240.000000,1000002.0,1.0
2,16.990000,10.250000,16.990000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,-4.250000,...,N,U,Y,U,U,U,Y,534.747973,1000003.0,3.0
3,38.000000,7.500000,38.000000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,-1.500000,...,U,Y,U,U,U,U,Y,534.744754,1000004.0,5.0
4,55.230000,570.500000,71.980000,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,38.500000,...,I,U,U,U,U,U,Y,434.000000,1000005.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,58.719985,513.559937,46.179136,0.888828,41.072247,13.55956,13.295062,0.0,0.047057,-13.933818,...,S,U,U,U,Y,U,Y,773.000000,1099996.0,6.0
99996,58.719985,513.559937,46.179136,0.888828,41.072247,13.55956,13.295062,0.0,0.047057,-13.933818,...,N,U,U,Y,Y,Y,Y,835.000000,1099997.0,6.0
99997,58.719985,513.559937,46.179136,0.888828,41.072247,13.55956,13.295062,0.0,0.047057,-13.933818,...,U,Y,Y,U,U,U,N,433.000000,1099998.0,3.0
99998,58.719985,513.559937,46.179136,0.888828,41.072247,13.55956,13.295062,0.0,0.047058,-13.933818,...,S,U,U,U,U,U,N,75.000000,1099999.0,2.0


In [52]:
count_outliers(knn_outlier_df, numeric_cols)

,Feature,Number of Outliers,Outlier Percentage
71,adults,31102,31.102
10,change_rev,26669,26.669
72,income,18923,18.923
27,threeway_Mean,17901,17.901
25,cc_mou_Mean,12750,12.750
...,...,...,...
12,drop_dat_Mean,0,0.000
14,blck_dat_Mean,0,0.000
16,unan_dat_Mean,0,0.000
74,Customer_ID,0,0.000


In [56]:
null_df.to_csv('../data/processed/cleaned_data.csv', index=False)